In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

# Replace with your credentials
user = "root"
password = "123456789"
host = "localhost"
port = 3306
database = "pollution_db"

# Create SQLAlchemy engine (using mysql-connector)
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")

# Now read a full table into pandas
df = pd.read_sql("SELECT * FROM pollution_data", con=engine)


In [2]:

# -----------------------------
# 1️⃣ Define Subindex Calculator
# -----------------------------
def calculate_subindex(concentration, breakpoints):
    for (C_lo, C_hi, I_lo, I_hi) in breakpoints:
        if C_lo <= concentration <= C_hi:
            return ((I_hi - I_lo) / (C_hi - C_lo)) * (concentration - C_lo) + I_lo
    return np.nan


# -----------------------------
# 2️⃣ Define CPCB Breakpoints
# -----------------------------
breakpoints = {
    'pm25': [
        (0, 30, 0, 50),
        (31, 60, 51, 100),
        (61, 90, 101, 200),
        (91, 120, 201, 300),
        (121, 250, 301, 400),
        (251, 500, 401, 500)
    ],
    'pm10': [
        (0, 50, 0, 50),
        (51, 100, 51, 100),
        (101, 250, 101, 200),
        (251, 350, 201, 300),
        (351, 430, 301, 400),
        (431, 500, 401, 500)
    ],
    'so2': [
        (0, 40, 0, 50),
        (41, 80, 51, 100),
        (81, 380, 101, 200),
        (381, 800, 201, 300),
        (801, 1600, 301, 400),
        (1601, 2000, 401, 500)
    ],
    'no2': [
        (0, 40, 0, 50),
        (41, 80, 51, 100),
        (81, 180, 101, 200),
        (181, 280, 201, 300),
        (281, 400, 301, 400),
        (401, 500, 401, 500)
    ],
    'co': [
        (0, 1, 0, 50),
        (1.1, 2, 51, 100),
        (2.1, 10, 101, 200),
        (10.1, 17, 201, 300),
        (17.1, 34, 301, 400),
        (34.1, 50, 401, 500)
    ],
    'o3': [
        (0, 50, 0, 50),
        (51, 100, 51, 100),
        (101, 168, 101, 200),
        (169, 208, 201, 300),
        (209, 748, 301, 400),
        (749, 1000, 401, 500)
    ]
}


# -----------------------------
# 3️⃣ Calculate Subindices
# -----------------------------
pollutants = ['pm25', 'pm10', 'so2', 'no2', 'co', 'o3']

for p in pollutants:
    if p in df.columns:
        df[f'{p}_subindex'] = df[p].apply(lambda x: calculate_subindex(x, breakpoints[p]))


# -----------------------------
# 4️⃣ Calculate Overall AQI
# -----------------------------
subindex_cols = [col for col in df.columns if col.endswith('_subindex')]
df['AQI'] = df[subindex_cols].max(axis=1)


# -----------------------------
# 5️⃣ Assign AQI Category
# -----------------------------
def get_aqi_category(aqi):
    if pd.isna(aqi):
        return np.nan
    elif aqi <= 50:
        return 'Good'
    elif aqi <= 100:
        return 'Satisfactory'
    elif aqi <= 200:
        return 'Moderate'
    elif aqi <= 300:
        return 'Poor'
    elif aqi <= 400:
        return 'Very Poor'
    else:
        return 'Severe'

df['AQI_Category'] = df['AQI'].apply(get_aqi_category)


In [3]:
df.to_csv('C:\\DATA ANALYSIS\\My projects\\polution-data\\12_final_cleaned_file.csv')